Concept: first 3 subfigs in the 1st row, second 2 subfigs in the 2nd row
* subfig 1) spatial distribution of the APD vector field across square-shaped surface for the case of TE polarization
* subfig 2) the unit normal vector field -||-
* subfig 3) normalized APD scalar field -||-
* subfig 4) APD scalar field mapped to a 2-dimensional parametrized surface for averaging -||-
* subfig 5) spatial distribution of the integration points -||-

In [ ]:
import os

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
import numpy as np
try:
    import open3d as o3d
except ImportError:
    import sys
    print(sys.exc_info())
import quadpy
from scipy import interpolate

from dosipy.utils.dataloader import load_ear_data
from dosipy.utils.viz import (set_colorblind, scatter_2d, scatter_3d,
                              fig_config, save_fig)
from helpers import (ABSTRACT_ID, clean_df, export_pcd, export_fields,
                     poynting_vector, export_rect_idx)

In [ ]:
mode = 'te'
frequency = 60
df = load_ear_data(mode, frequency)
df = clean_df(df)
xyz = export_pcd(df)
E, H = export_fields(df)
Sx, Sy, Sz = poynting_vector(E, H)

In [ ]:
# crop the excess from an ear model
crop_idxs = np.where(xyz[:, 0] > 67)[0]
xyz_crop = xyz[crop_idxs]
Sx_crop, Sy_crop, Sz_crop = Sx[crop_idxs], Sy[crop_idxs], Sz[crop_idxs]
pcd_crop = o3d.geometry.PointCloud()
pcd_crop.points = o3d.utility.Vector3dVector(xyz_crop)

# move to the center
center_crop = pcd_crop.get_center()
xyz_crop_t = np.c_[xyz_crop[:, 0] - center_crop[0],
                   xyz_crop[:, 1] - center_crop[1],
                   xyz_crop[:, 2] - center_crop[2]]
pcd_crop_t = o3d.geometry.PointCloud()
pcd_crop_t.points = o3d.utility.Vector3dVector(xyz_crop_t)

# extract visible points across xy-plane
diameter = np.linalg.norm(
    pcd_crop_t.get_max_bound() - pcd_crop_t.get_min_bound()
)
radius = 10 ** 5.5
camera = [0, 0, -diameter]
_, pt_map = pcd_crop_t.hidden_point_removal(camera, radius)
xyz_crop_t_xy = xyz_crop_t[pt_map]
Sx_crop_t_xy = Sx_crop[pt_map]
Sy_crop_t_xy = Sy_crop[pt_map]
Sz_crop_t_xy = Sz_crop[pt_map]
pcd_crop_t_xy = o3d.geometry.PointCloud()
pcd_crop_t_xy.points = o3d.utility.Vector3dVector(xyz_crop_t_xy)
pcd_crop_t_xy.estimate_normals()
pcd_crop_t_xy.orient_normals_consistent_tangent_plane(30)
n_crop_t_xy = np.asarray(pcd_crop_t_xy.normals)

# extract only the points that are found on the averaging surface
avg_center = [0.21, 3.25]
edge_length = 10
area = edge_length ** 2
origin, idx_rect = export_rect_idx(xyz=xyz_crop_t_xy,
                                   center=avg_center,
                                   edge_length=edge_length,
                                   view='xy')
xyz_rect = xyz_crop_t_xy[idx_rect]
Sx_rect = Sx_crop_t_xy[idx_rect]
Sy_rect = Sy_crop_t_xy[idx_rect]
Sz_rect = Sz_crop_t_xy[idx_rect]
n_rect = n_crop_t_xy[idx_rect]

In [ ]:
skip = 20  # downsample point cloud uniformly

set_colorblind()
fig_config(latex=True, scaler=1.75, text_size=20, line_width=1)
fig, ax = scatter_3d({'$x$ [mm]': xyz_rect[::skip, 0],
                      '$z$ [mm]': xyz_rect[::skip, 2],
                      '$y$ [mm]': xyz_rect[::skip, 1]},
                     elev=[20], azim=[-70])
ax.quiver(xyz_rect[::skip, 0], xyz_rect[::skip, 2], xyz_rect[::skip, 1],
          Sx_rect[::skip].real, Sz_rect[::skip].real, Sy_rect[::skip].real,
          normalize=True, length=2, arrow_length_ratio=0.3, alpha=1, color='b')
ax.set(xticks=[-4.79, 0.21, 5.21],
       yticks=[-18, -15, -12],
       zticks=[-1.7, 3.2, 8.3])
ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.xaxis.labelpad = 13
ax.yaxis.labelpad = 19
ax.zaxis.labelpad = 13
    
# fname = os.path.join('figures', f'fig_{ABSTRACT_ID}_3a')
# save_fig(fig, fname=fname, formats=['png'])

In [ ]:
set_colorblind()
fig_config(latex=True, scaler=1.75, text_size=20, line_width=1)
fig, ax = scatter_3d({'$x$ [mm]': xyz_rect[::skip, 0],
                      '$z$ [mm]': xyz_rect[::skip, 2],
                      '$y$ [mm]': xyz_rect[::skip, 1]},
                     elev=[20], azim=[-70])
ax.quiver(xyz_rect[::skip, 0], xyz_rect[::skip, 2], xyz_rect[::skip, 1],
          -n_rect[::skip, 0], -n_rect[::skip, 2], -n_rect[::skip, 1],
          normalize=True, length=2, arrow_length_ratio=0.3, alpha=1, color='r')
ax.set(xticks=[-4.79, 0.21, 5.21],
       yticks=[-18, -15, -12],
       zticks=[-1.7, 3.2, 8.3])
ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.xaxis.labelpad = 13
ax.yaxis.labelpad = 19
ax.zaxis.labelpad = 13

# fname = os.path.join('figures', f'fig_{ABSTRACT_ID}_3b')
# save_fig(fig, fname=fname, formats=['png'])

In [ ]:
S = (Sx_rect[::skip].real * n_rect[::skip, 0]
     + Sz_rect[::skip].real * n_rect[::skip, 2]
     + Sy_rect[::skip].real * n_rect[::skip, 1])

set_colorblind()
fig_config(latex=True, scaler=1.75, text_size=20, line_width=1)
fig, ax = scatter_3d({'$x$ [mm]': xyz_rect[::skip, 0],
                      '$z$ [mm]': xyz_rect[::skip, 2],
                      '$y$ [mm]': xyz_rect[::skip, 1],
                      '$APD$': S},
                     elev=[20], azim=[-70])
ax.set(xticks=[-4.79, 0.21, 5.21],
       yticks=[-18, -15, -12],
       zticks=[-1.7, 3.2, 8.3])
ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.xaxis.labelpad = 13
ax.yaxis.labelpad = 19
ax.zaxis.labelpad = 13

# fname = os.path.join('figures', f'fig_{ABSTRACT_ID}_3c')
# save_fig(fig, fname=fname, formats=['png'])

In [ ]:
S = (Sx_rect[::skip].real * n_rect[::skip, 0]
     + Sz_rect[::skip].real * n_rect[::skip, 2]
     + Sy_rect[::skip].real * n_rect[::skip, 1])

set_colorblind()
fig_config(latex=True, scaler=1.5, text_size=20, line_width=1)
fig, ax = scatter_2d({'$x$ [mm]': xyz_rect[::skip, 0],
                      '$y$ [mm]': xyz_rect[::skip, 1],
                      '$APD$': S}, s=40)
ax.set(xticks=[-4.79, 0.21, 5.21],
       yticks=[-1.7, 3.2, 8.3]);

# fname = os.path.join('figures', f'fig_{ABSTRACT_ID}_3d')
# save_fig(fig, fname=fname, formats=['png'])

In [ ]:
def visualize_integration_scheme(patch, pts, radii, colors):
    """Return figure and axis of the integration scheme.

    Parameters
    ----------
    patch : matplotlib.patches.Patch
        A 2-D artist with a face and an edge color.
    pts : numpy.ndarray
        Array of points over which the integration will be performed.
    radii : numpy.ndarray
        Array of radii that defines the weight for each corresponding
        integration point.
    colors : list
        Color of the circular patch that defines the area of influence
        at each integration point.

    Returns
    -------
    tuple
        Figure and axis.
    """
    set_colorblind()
    fig_config(latex=True, scaler=1.5, text_size=20, line_width=1)
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.axis("equal")
    ax.set_axis_off()
    ax.add_patch(patch)
    for pt, radius, color in zip(pts, radii, colors):
        circ = Circle((pt[0], pt[1]), radius, ec='gray', fc='none')
        ax.add_patch(circ)
    plt.scatter(pts[:, 0], pts[:, 1], marker='o', s=60, c=colors, cmap='viridis')
    return fig, ax


bbox = [xyz_rect[:, 0].min(), xyz_rect[:, 0].max(),
        xyz_rect[:, 1].min(), xyz_rect[:, 1].max()]
func = interpolate.Rbf(xyz_rect[::skip, 0], xyz_rect[::skip, 1], S)
x_a, x_b, y_a, y_b = bbox
r, w = np.polynomial.legendre.leggauss(6)
x = 0.5 * (r + 1.) * (x_b - x_a) + x_a
y = 0.5 * (r + 1.) * (y_b - y_a) + y_a
w = 0.5 * w * (bnd_r - bnd_l)
Xx, Xy = np.meshgrid(x, y)
Wx, Wy = np.meshgrid(w, w)
pts = np.c_[Xx.ravel(), Xy.ravel()]
wts = np.c_[Wx.ravel(), Wy.ravel()]
wts_norm = np.linalg.norm(wts, axis=1, ord=sum(abs(x)**2)**(1./2))
area_tot = (x_b - x_a) * (y_b - y_a)
radii = np.sqrt(abs(wts_norm) / np.sum(wts_norm) * area_tot / (np.pi))
colors = func(Xx, Xy).ravel()
rect = Rectangle((x_a, y_a), x_b-x_a, y_b-y_a, ec='k', fc='None', lw=2)
fig, ax = visualize_integration_scheme(rect, pts, radii, colors)

# fname = os.path.join('figures', f'fig_{ABSTRACT_ID}_3e')
# save_fig(fig, fname=fname, formats=['png'])